In [1]:
import torch
import numpy as np
import csv,os
import xlwings as xw
from Spectral_preprocess import airpls,SG,max_min 
import torch

In [2]:
app = xw.App(visible=False, add_book=False)
workbook = app.books.open('original-data.xlsx')
sheets_count = workbook.sheets.count
data=[]
for i in range(sheets_count):
    worksheet=workbook.sheets[i]
    data_ = worksheet.range('B1').expand().options(np.array).value
    m,n = data_.shape
    #print(m,n)
    label_=np.array(list(worksheet.name),dtype=int)
    k=int(label_.shape[0])
    label_=np.ones(25*k).reshape(k,25).T*label_.T
    data_=data_[:,:25]
    if i==0:
        data=data_
        label=label_
    else:
        data=np.concatenate((data,data_),axis=1)
        label=np.concatenate((label,label_), axis=0)
data=data.T
#print(data.shape,label.shape)
workbook.close()
app.quit()

In [4]:
type(data[0,0])

numpy.float64

In [28]:
#data=SG(data)
#data=airpls(data)
data=data[:,160:783]
data=max_min(data)
data = data.reshape(31, 25, 623).swapaxes(0, 1).reshape(775, 1, 623)
label = label.reshape(31, 25, 5).swapaxes(0, 1).reshape(775, 5)
data_train =torch.from_numpy( data[:int(0.7 * len(data))])
label_train =torch.from_numpy( label[:int(0.7 * len(label))])
data_val= torch.from_numpy( data[int(0.7 * len(data)):int(0.8 * len(data))])
label_val =torch.from_numpy( label[int(0.7 * len(label)):int(0.8 * len(label))])
data_test= torch.from_numpy( data[int(0.8 * len(data)):])
label_test=torch.from_numpy( label[int(0.8 * len(label)):])
torch.save((data_train,label_train),'train.pt')
torch.save((data_val,label_val),'val.pt')
torch.save((data_test,label_test),'test.pt')